# Denoising experiments

## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn

## Step 1: Data Analysis

In [ ]:
df = pd.read_csv("OpenBCI-RAW.txt", comment='%')
df = df.rename(columns=lambda x: x.strip())
df = df[5:]
df.head()

In [1]:
import matplotlib.pyplot as plt

# Define the applications and their coordinates on the impact and frequency axes
applications = {
    "Medical": (8, 6),
    "Gaming and Entertainment": (5, 5),
    "Mental Health": (7, 4),
    "Education and Training": (5, 3),
    "Research": (9, 2),
    "Security": (7, 2),
    "Communication": (8, 2)
}

# Create lists for impact and frequency
impact = [applications[app][0] for app in applications]
frequency = [applications[app][1] for app in applications]
labels = list(applications.keys())

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(frequency, impact, color='b')

# Annotate the points
for i, label in enumerate(labels):
    plt.text(frequency[i], impact[i], label, fontsize=9, ha='right')

# Set chart title and labels
plt.title('BCI Applications: Impact vs. Frequency')
plt.xlabel('Frequency')
plt.ylabel('Impact')
plt.grid(True)
plt.show()

In [ ]:
df.columns

In [ ]:
def plot_eeg(data, figsize=(20, 15), title=None):
    if isinstance(data, torch.Tensor):
        data = data.detach().numpy()

    num_plots = data.shape[1]

    fig, ax = plt.subplots(num_plots, 1, figsize=figsize)
    x = range(len(data))

    if data.ndim == 1:
        data = data[:, None]

    if num_plots == 1:
        ax = [ax]
        # data = data.reshape(1, -1)

    for i in range(num_plots):
        ax[i].plot(x, data[:, i], linewidth=0.5)
        ax[i].set_xlabel("Iter")
        ax[i].set_ylabel("Voltage ($\mu V$)")
        if title:
            ax[i].set_title(title)
        else:
            ax[i].set_title(df.columns[start_idx+i])

    plt.tight_layout()
    plt.show()

In [ ]:
start_idx = 1
end_idx = start_idx + 8
raw_data = df.iloc[:, start_idx:end_idx].to_numpy()
plot_eeg(raw_data)

## 1. 1D Convolutions

In [ ]:
data_conv_df = df.iloc[:, 1]
data_conv_df.head()

In [ ]:
data_conv = torch.tensor(data_conv_df.to_numpy(), dtype=torch.float32).view(1, 1, -1)

In [ ]:
class ConvolutionNoiseReductor(nn.Module):
    def __init__(self, kernel_size, stride=None, padding=0):
        super(ConvolutionNoiseReductor, self).__init__()
        self.conv = nn.Conv1d(1, 1, kernel_size=kernel_size, stride=stride, padding=padding)

    def forward(self, x):
        x = self.conv(x)
        return x

In [ ]:
model = ConvolutionNoiseReductor(kernel_size=256, stride=16)

result = model(data_conv)

In [ ]:
plot_eeg(df["EXG Channel 0"].to_numpy()[..., np.newaxis], figsize=(15, 3), title="Original")
plot_eeg(result[0, :, :].T, figsize=(15, 3), title="Denoinsed")

**Convolutions ❌** \\
One major flaw of using convolutions for EEG signal denoising is that the values have a very large shift after applying the convolution.

## 2. ICA

In [ ]:
!pip install -q mne

In [ ]:
import mne
from mne.preprocessing import ICA

In [ ]:
data_ica_df = df.iloc[:, 1:10]
data_ica_df.head()

In [ ]:
sfreq = 100
info = mne.create_info(ch_names=data_ica_df.columns.tolist(), sfreq=sfreq, ch_types='eeg')

raw = mne.io.RawArray(data_ica_df.T.values, info)

ica = ICA(n_components=None, random_state=42, max_iter=800)
raw.filter(l_freq=1.0, h_freq=None)
ica.fit(raw)

In [ ]:
ica.apply(raw)
denoised_df = pd.DataFrame(raw.get_data().T, columns=data_ica_df.columns.tolist())
denoised_df.head()

In [ ]:
plot_eeg(df["EXG Channel 0"].to_numpy()[..., np.newaxis], figsize=(15, 3), title="Original")
plot_eeg(denoised_df.to_numpy(), title="Denoinsed")